In [110]:
import re
import json
from collections import defaultdict

import pandas as pd
import matplotlib.pyplot as plt

from transformers import AutoTokenizer

In [54]:
def percentile(n):
    def percentile_(x):
        return x.quantile(n)
    percentile_.__name__ = 'percentile_{:02.0f}'.format(n*100)
    
    return percentile_

In [55]:
model = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
train_data = pd.read_csv("../../data/train.csv")

## 레이블 별 데이터 분포

In [53]:
train_data['target'].value_counts()

target
1    1000
0    1000
3    1000
4    1000
5    1000
6    1000
2    1000
Name: count, dtype: int64

## 데이터 샘플 수

In [57]:
print("총 데이터 샘플 수 =", len(train_data))
print("레이블 개수 =", train_data['target'].nunique())

print("중복되는 샘플 수 =", sum(train_data.duplicated(keep=False, subset=['text'])))
print("중복 제외 샘플 수 =", train_data['text'].nunique())

총 데이터 샘플 수 = 7000
레이블 개수 = 7
중복되는 샘플 수 = 0
중복 제외 샘플 수 = 7000


## 단어 단위 문장 길이

In [58]:
print("단어 단위 문장 길이(95 percentile) =",\
      train_data['text'].apply(lambda x: len(x.split(' '))).agg([percentile(0.95)]).values[0])

train_data['len'] = train_data['text'].apply(lambda x: len(x.split(' ')))
print(train_data.agg({'len':['min', 'max', 'mean']}))

단어 단위 문장 길이(95 percentile) = 9.0
            len
min    2.000000
max   13.000000
mean   6.545571


## 토큰 단위 문장 길이

In [61]:
print("토큰 단위 문장 길이(95 percentile) =",\
      train_data['text'].apply(lambda x: len(tokenizer(x)['input_ids'])).agg([percentile(0.95)]).values[0])

train_data['token_len'] = train_data['text'].apply(lambda x: len(tokenizer(x)['input_ids']))
print(train_data.agg({'token_len':['min', 'max', 'mean']}))

토큰 단위 문장 길이(95 percentile) = 21.0
      token_len
min    4.000000
max   29.000000
mean  15.547571


## klue/bert-base가 알고 있는 특수 문자들

In [130]:
not_en_ko_num_pattern = re.compile('[^ㄱ-ㅎ가-힣a-zA-Z0-9\s]')
known_special_chars_dict = defaultdict(int)

for sentence in train_data['text']:
    special_chars = not_en_ko_num_pattern.findall(sentence)
    if len(special_chars) != 0 :
        for char in special_chars:
            if tokenizer.tokenize(char) != ['[UNK]']:
                known_special_chars_dict[char] += 1

print("klue/bert-base가 알고 있는 특수문자 개수 =", len(known_special_chars_dict))
print(known_special_chars_dict)

klue/bert-base가 알고 있는 특수문자 개수 = 76
defaultdict(<class 'int'>, {'·': 1135, '…': 2955, '.': 678, '%': 226, '↑': 70, '美': 218, '北': 203, '與': 31, '佛': 15, '外': 5, '人': 2, '朴': 97, '野': 23, 'ｍ': 4, '中': 112, '日': 55, '文': 29, '獨': 22, '⑤': 2, '∼': 61, '黨': 2, '⑥': 1, '靑': 52, '法': 1, '韓': 23, '女': 10, '前': 16, '詩': 5, '英': 36, '㎝': 11, '孫': 3, '→': 25, '①': 5, '反': 5, '㈜': 4, '對': 6, '黃': 4, '故': 2, '㎜': 11, '重': 2, 'ㆍ': 11, '④': 3, '軍': 9, '③': 5, '銀': 3, '％': 7, '族': 1, '安': 10, '證': 1, '金': 4, '②': 6, '㎞': 6, '㎡': 2, '車': 3, '大': 2, '戰': 1, '南': 5, '富': 1, '新': 1, '＋': 5, '行': 5, '親': 3, '‘': 1, '無': 2, '㎏': 1, '四': 1, '體': 1, '硏': 4, '色': 1, '强': 1, '℃': 1, '風': 2, '父': 2, '○': 4, '×': 1, '社': 1})


## klue/bert-base가 모르는 특수 문자들

In [131]:
not_en_ko_num_pattern = re.compile('[^ㄱ-ㅎ가-힣a-zA-Z0-9\s]')
unk_special_chars_dict = defaultdict(int)

for sentence in train_data['text']:
    special_chars = not_en_ko_num_pattern.findall(sentence)
    if len(special_chars) != 0 :
        for char in special_chars:
            tokenizer.tokenize(char)
            if tokenizer.tokenize(char) == ['[UNK]']:
                unk_special_chars_dict[char] += 1

print("klue/bert-base가 모르는 특수문자 개수 =", len(unk_special_chars_dict))
print(unk_special_chars_dict)


klue/bert-base가 모르는 특수문자 개수 = 36
defaultdict(<class 'int'>, {'⅔': 6, '脫': 2, '伊': 18, '亞': 6, '⅓': 2, '↓': 33, '檢': 8, '展': 6, '曺': 2, '↔': 2, '企': 1, '核': 2, '㎛': 2, '秋': 1, '弗': 3, '潘': 1, '比': 2, '蘭': 1, '樂': 1, '勝': 1, '崔': 1, '號': 1, '₂': 1, '禹': 1, '⑫': 1, 'ｇ': 1, '駐': 1, '寒': 1, '賞': 1, '尹': 1, '港': 1, '印': 1, '⑪': 1, '千': 1, '協': 1, '㎓': 1})
